# Intelligent Sampler Demo

In [ ]:
import os
import pandas as pd
import numpy as np
import requests

In [ ]:
from sandag_rsm.sampler import rsm_household_sampler

### Remote I/O

In [ ]:
data_dir = "./data-dl/"
os.makedirs(data_dir, exist_ok=True)

resource_url = "https://media.githubusercontent.com/media/wsp-sag/client_sandag_rsm_resources/main/"

mgra_filename = "mgra13_based_input2016.csv.gz"
skim_filename = "traffic_skims_AM_mini.omx"
trips_filename = "trips_sample.pq"
download_files_vector = [
    "hh_test.csv.gz", 
    "person_test.csv.gz", 
    "taz_crosswalk.csv", 
    "accessibilities.csv", 
    "accessibilities_iter.csv", 
]

# for download_file in download_files_vector:
#     r = requests.get((resource_url+download_file), allow_redirects=True)
#     open((data_dir+download_file), 'wb').write(r.content)

### Temp directory for output

In [ ]:
import tempfile
tempdir = tempfile.TemporaryDirectory()
tempdir.name

## Original Full Household File
This file has the original model household data, not merged TAZs.

In [ ]:
h0 = pd.read_csv("~/LocalGit/client_sandag_rsm_resources/hh_test.csv.gz")

## Sampling without Accessilibility Changes

In [ ]:
h1, p1 = rsm_household_sampler(
    input_dir="./data-dl",
    output_dir=tempdir.name,
    input_household="hh_test.csv.gz",
    input_person="person_test.csv.gz",
    taz_crosswalk="taz_crosswalk.csv",
    output_household="sampled_households_1.csv",
    output_person="sampled_person_1.csv",
)

## Sampling with Accessilibility Changes

In [ ]:
h2, p2 = rsm_household_sampler(
    input_dir="~/LocalGit/client_sandag_rsm_resources",
    output_dir=tempdir.name,
    prev_iter_access="accessibilities.csv",
    curr_iter_access="accessibilities_iter.csv",
    input_household="hh_test.csv.gz",
    input_person="person_test.csv.gz",
    taz_crosswalk="taz_crosswalk.csv",
    output_household="sampled_households_2.csv",
    output_person="sampled_person_2.csv",
)

## Sampling with Study Area Defined

In [ ]:
h3, p3 = rsm_household_sampler(
    input_dir="~/LocalGit/client_sandag_rsm_resources",
    output_dir=tempdir.name,
    prev_iter_access="accessibilities.csv",
    curr_iter_access="accessibilities_iter.csv",
    input_household="hh_test.csv.gz",
    input_person="person_test.csv.gz",
    taz_crosswalk="taz_crosswalk.csv",
    output_household="sampled_households_3.csv",
    output_person="sampled_person_3.csv",
    study_area=[49,50],
)